# Imports

In [50]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [51]:
from VAE1D import *

In [52]:
from IPython.core.debugger import set_trace

Define train function in notebook due to import namespace bug.

In [4]:
def train_VAE1D(dl):
    """
    Execute the training loop
    """
    loss_tracker = AvgTracker()
    kl_tracker = AvgTracker()
    logp_tracker = AvgTracker()
    timer = StopWatch()
    freq = min(log_freq, len(dl))
    
    for i, (X, _) in enumerate(tqdm(dl)):
        
        X = X.to(device)
        timer.lap()  # load time
        
        # Generate transient and compute loss
        X_hat, mu, logvar = model(X)
        loss, loss_desc = criterion(X_hat, X, mu, logvar)
        timer.lap()  # gen time
        
        loss_tracker.update(loss.item())
        kl_tracker.update(loss_desc['KL'].item())
        logp_tracker.update(loss_desc['logp'].item())
        
        if model.training:
            # Update weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            timer.lap()  # backprop time
        
        if ((i + 1) % freq == 0) and ((epoch + 1) % ep_freq == 0):
            # Print progress
            if model.training:
                print('TRAINING')
            else:
                print('VALIDATION')
            print(f'Epoch: {epoch + 1} ({i + 1}/{len(dl)})')
            print(f'\tData load time: {timer.elapsed[0]:.3f} sec')
            print(f'\tGeneration time: {timer.elapsed[1]:.3f} sec')
            if model.training:
                print(f'\tBackprop time: {timer.elapsed[2]:.3f} sec')
            print(f'\tLog probability: {logp_tracker.val:.4f} '
                  f'(avg {logp_tracker.avg:.4f})')
            print(f'\tKL: {kl_tracker.val:.4f} (avg {kl_tracker.avg:.4f})')
            print(f'\tLoss: {loss_tracker.val:.4f} (avg {loss_tracker.avg:.4f})')

    return loss_tracker.avg, kl_tracker.avg, logp_tracker.avg

# Accumulator

### Build model and loss function 
Initially designed for 2D input images, modified for 1D time-series data.
Based on this paper: https://arxiv.org/abs/1807.01349

In [5]:
# The hydraulic system has 14 sensors from which to pull data
n_channels = 14
# The data has been resized to 512, although it represents 1 min for each cycle
size = 512
# Latent space is restricted to be about 1/170th of the input dims, similar to 2D case
n_latent = 50

In [6]:
model = VAE1D(size, n_channels, n_latent)

In [7]:
beta = 1  # KL term relative weight
criterion = VAE1DLoss(beta)

### Load hydraulic test data
From this dataset: https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems#

In [8]:
desc = 'accumulator'
data_path = Path(f'data/hydraulic/{desc}/')
batch_size = 32
train_dl, val_dl, test_dl = load_datasets(data_path,
                                          batch_size=batch_size)

In [9]:
print(len(train_dl), len(val_dl), len(test_dl))

27 7 1157


### Prepare for training

In [10]:
# Display settings - TODO add Visdom logging
log_freq = 15 # batches
ep_freq = 10  # epochs

# Training parameters
epochs = 300
lr = 1e-3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Checkpoint to resume from (default None)
load_path = None

# Checkpoint save location
save_path = Path(f"models/{date.today().strftime('%y%m%d')}-{desc}/")
if save_path.is_dir():
    print(f"Folder {save_path} already exists")
else:
    os.mkdir(save_path)
save_path

Folder models/190130-accumulator already exists


PosixPath('models/190130-accumulator')

In [11]:
# Load checkpoint if any
if load_path is not None:
    checkpoint = torch.load(load_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("Checkpoint loaded")
    print(f"Validation loss: {checkpoint['val_loss']}")
    print(f"Epoch: {checkpoint['epoch']}")

In [12]:
# Load optimizer and scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10)

In [13]:
# Move to GPU
model = model.to(device)
criterion = criterion.to(device)

### Train the model

In [14]:
# Main loop
best_loss = np.inf
for epoch in range(epochs):

    model.train()
    scheduler.step()
    train_loss, train_kl, train_logp = train_VAE1D(train_dl)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_kl, val_logp = train_VAE1D(val_dl)

    # Report training progress to user
    if val_loss < best_loss:
        print('Saving checkpoint..')
        best_loss = val_loss
        save_dict = {'epoch': epoch + 1,
                     'state_dict': model.state_dict(),
                     'val_loss': val_loss,
                     'optimizer': optimizer.state_dict()}
        path = save_path / f'best_model-{n_latent}-{beta}.pt'
        torch.save(save_dict, path)
        print(f'Lowest validation loss: {best_loss:.4f}')

  0%|          | 0/27 [00:00<?, ?it/s]

Saving checkpoint..
Lowest validation loss: 583.5805


  0%|          | 0/27 [00:00<?, ?it/s]

Saving checkpoint..
Lowest validation loss: 536.5839


  0%|          | 0/27 [00:00<?, ?it/s]

...

VALIDATION
Epoch: 290 (7/7)
	Data load time: 0.139 sec
	Generation time: 0.146 sec
	Log probability: -3.0173 (avg -2.8356)
	KL: 23.1499 (avg 19.3533)
	Loss: 26.1671 (avg 22.1890)


 70%|███████   | 19/27 [00:00<00:00, 18.23it/s]

Epoch: 300 (15/27)
	Data load time: 0.145 sec
	Generation time: 0.157 sec
	Backprop time: 0.176 sec
	Log probability: -7.3365 (avg -4.8723)
	KL: 38.9763 (avg 59.1629)
	Loss: 46.3128 (avg 64.0351)


100%|██████████| 7/7 [00:00<00:00, 24.97it/s]

VALIDATION
Epoch: 300 (7/7)
	Data load time: 0.142 sec
	Generation time: 0.148 sec
	Log probability: -5.1778 (avg -4.5715)
	KL: 26.1836 (avg 27.0229)
	Loss: 31.3614 (avg 31.5944)


# Cooler

### Build model and loss function 
Initially designed for 2D input images, modified for 1D time-series data.
Based on this paper: https://arxiv.org/abs/1807.01349

In [15]:
# The hydraulic system has 14 sensors from which to pull data
n_channels = 14
# The data has been resized to 512, although it represents 1 min for each cycle
size = 512
# Latent space is restricted to be about 1/170th of the input dims, similar to 2D case
n_latent = 50

In [16]:
model = VAE1D(size, n_channels, n_latent)

In [17]:
beta = 1  # KL term relative weight
criterion = VAE1DLoss(beta)

### Load hydraulic test data
From this dataset: https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems#

In [18]:
desc = 'cooler'
data_path = Path(f'data/hydraulic/{desc}/')
batch_size = 32
train_dl, val_dl, test_dl = load_datasets(data_path,
                                          batch_size=batch_size)

In [19]:
print(len(train_dl), len(val_dl), len(test_dl))

28 7 1100


### Prepare for training

In [20]:
# Display settings - TODO add Visdom logging
log_freq = 15 # batches
ep_freq = 10  # epochs

# Training parameters
epochs = 300
lr = 1e-3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Checkpoint to resume from (default None)
load_path = None

# Checkpoint save location
save_path = Path(f"models/{date.today().strftime('%y%m%d')}-{desc}/")
if save_path.is_dir():
    print(f"Folder {save_path} already exists")
else:
    os.mkdir(save_path)
save_path

Folder models/190130-cooler already exists


PosixPath('models/190130-cooler')

In [21]:
# Load checkpoint if any
if load_path is not None:
    checkpoint = torch.load(load_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("Checkpoint loaded")
    print(f"Validation loss: {checkpoint['val_loss']}")
    print(f"Epoch: {checkpoint['epoch']}")

In [22]:
# Load optimizer and scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10)

In [23]:
# Move to GPU
model = model.to(device)
criterion = criterion.to(device)

### Train the model

In [24]:
# Main loop
best_loss = np.inf
for epoch in range(epochs):

    model.train()
    scheduler.step()
    train_loss, train_kl, train_logp = train_VAE1D(train_dl)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_kl, val_logp = train_VAE1D(val_dl)

    # Report training progress to user
    if val_loss < best_loss:
        print('Saving checkpoint..')
        best_loss = val_loss
        save_dict = {'epoch': epoch + 1,
                     'state_dict': model.state_dict(),
                     'val_loss': val_loss,
                     'optimizer': optimizer.state_dict()}
        path = save_path / f'best_model-{n_latent}-{beta}.pt'
        torch.save(save_dict, path)
        print(f'Lowest validation loss: {best_loss:.4f}')

  0%|          | 0/28 [00:00<?, ?it/s]

Saving checkpoint..
Lowest validation loss: 519.6896


  0%|          | 0/28 [00:00<?, ?it/s]

Saving checkpoint..
Lowest validation loss: 234.6378


  0%|          | 0/28 [00:00<?, ?it/s]

...

Saving checkpoint..
Lowest validation loss: 6.1716


 68%|██████▊   | 19/28 [00:00<00:00, 18.27it/s]

Epoch: 300 (15/28)
	Data load time: 0.148 sec
	Generation time: 0.159 sec
	Backprop time: 0.178 sec
	Log probability: -3.1056 (avg -3.8799)
	KL: 10.6112 (avg 12.7134)
	Loss: 13.7168 (avg 16.5933)


100%|██████████| 7/7 [00:00<00:00, 24.73it/s]

VALIDATION
Epoch: 300 (7/7)
	Data load time: 0.136 sec
	Generation time: 0.145 sec
	Log probability: -1.7787 (avg -1.8827)
	KL: 16.4448 (avg 17.6002)
	Loss: 18.2235 (avg 19.4829)


# Pump

### Build model and loss function 
Initially designed for 2D input images, modified for 1D time-series data.
Based on this paper: https://arxiv.org/abs/1807.01349

In [25]:
# The hydraulic system has 14 sensors from which to pull data
n_channels = 14
# The data has been resized to 512, although it represents 1 min for each cycle
size = 512
# Latent space is restricted to be about 1/170th of the input dims, similar to 2D case
n_latent = 50

In [26]:
model = VAE1D(size, n_channels, n_latent)

In [27]:
beta = 1  # KL term relative weight
criterion = VAE1DLoss(beta)

### Load hydraulic test data
From this dataset: https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems#

In [28]:
desc = 'pump'
data_path = Path(f'data/hydraulic/{desc}/')
batch_size = 32
train_dl, val_dl, test_dl = load_datasets(data_path,
                                          batch_size=batch_size)

In [29]:
print(len(train_dl), len(val_dl), len(test_dl))

33 9 920


### Prepare for training

In [30]:
# Display settings - TODO add Visdom logging
log_freq = 15 # batches
ep_freq = 10  # epochs

# Training parameters
epochs = 300
lr = 1e-3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Checkpoint to resume from (default None)
load_path = None

# Checkpoint save location
save_path = Path(f"models/{date.today().strftime('%y%m%d')}-{desc}/")
if save_path.is_dir():
    print(f"Folder {save_path} already exists")
else:
    os.mkdir(save_path)
save_path

Folder models/190130-pump already exists


PosixPath('models/190130-pump')

In [31]:
# Load checkpoint if any
if load_path is not None:
    checkpoint = torch.load(load_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("Checkpoint loaded")
    print(f"Validation loss: {checkpoint['val_loss']}")
    print(f"Epoch: {checkpoint['epoch']}")

In [32]:
# Load optimizer and scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10)

In [33]:
# Move to GPU
model = model.to(device)
criterion = criterion.to(device)

### Train the model

In [34]:
# Main loop
best_loss = np.inf
for epoch in range(epochs):

    model.train()
    scheduler.step()
    train_loss, train_kl, train_logp = train_VAE1D(train_dl)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_kl, val_logp = train_VAE1D(val_dl)

    # Report training progress to user
    if val_loss < best_loss:
        print('Saving checkpoint..')
        best_loss = val_loss
        save_dict = {'epoch': epoch + 1,
                     'state_dict': model.state_dict(),
                     'val_loss': val_loss,
                     'optimizer': optimizer.state_dict()}
        path = save_path / f'best_model-{n_latent}-{beta}.pt'
        torch.save(save_dict, path)
        print(f'Lowest validation loss: {best_loss:.4f}')

  0%|          | 0/33 [00:00<?, ?it/s]

Saving checkpoint..
Lowest validation loss: 550.6035


  0%|          | 0/33 [00:00<?, ?it/s]

Saving checkpoint..
Lowest validation loss: 432.5109


  0%|          | 0/33 [00:00<?, ?it/s]

...

Epoch: 300 (15/33)
	Data load time: 0.143 sec
	Generation time: 0.156 sec
	Backprop time: 0.177 sec
	Log probability: -3.0672 (avg -4.4768)
	KL: 74.3558 (avg 64.2142)
	Loss: 77.4230 (avg 68.6910)


  0%|          | 0/9 [00:00<?, ?it/s]

Epoch: 300 (30/33)
	Data load time: 0.143 sec
	Generation time: 0.156 sec
	Backprop time: 0.177 sec
	Log probability: -7.9831 (avg -5.0632)
	KL: 49.1063 (avg 63.2834)
	Loss: 57.0894 (avg 68.3466)


100%|██████████| 9/9 [00:00<00:00, 28.35it/s]

VALIDATION
Epoch: 300 (9/9)
	Data load time: 0.145 sec
	Generation time: 0.155 sec
	Log probability: -13.1105 (avg -5.7936)
	KL: 491.0399 (avg 110.2611)
	Loss: 504.1504 (avg 116.0547)


# Valve

### Build model and loss function 
Initially designed for 2D input images, modified for 1D time-series data.
Based on this paper: https://arxiv.org/abs/1807.01349

In [35]:
# The hydraulic system has 14 sensors from which to pull data
n_channels = 14
# The data has been resized to 512, although it represents 1 min for each cycle
size = 512
# Latent space is restricted to be about 1/170th of the input dims, similar to 2D case
n_latent = 50

In [36]:
model = VAE1D(size, n_channels, n_latent)

In [37]:
beta = 1  # KL term relative weight
criterion = VAE1DLoss(beta)

### Load hydraulic test data
From this dataset: https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems#

In [38]:
desc = 'valve'
data_path = Path(f'data/hydraulic/{desc}/')
batch_size = 32
train_dl, val_dl, test_dl = load_datasets(data_path,
                                          batch_size=batch_size)

In [39]:
print(len(train_dl), len(val_dl), len(test_dl))

38 10 720


### Prepare for training

In [40]:
# Display settings - TODO add Visdom logging
log_freq = 15 # batches
ep_freq = 10  # epochs

# Training parameters
epochs = 300
lr = 1e-3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Checkpoint to resume from (default None)
load_path = None

# Checkpoint save location
save_path = Path(f"models/{date.today().strftime('%y%m%d')}-{desc}/")
if save_path.is_dir():
    print(f"Folder {save_path} already exists")
else:
    os.mkdir(save_path)
save_path

Folder models/190130-valve already exists


PosixPath('models/190130-valve')

In [41]:
# Load checkpoint if any
if load_path is not None:
    checkpoint = torch.load(load_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("Checkpoint loaded")
    print(f"Validation loss: {checkpoint['val_loss']}")
    print(f"Epoch: {checkpoint['epoch']}")

In [42]:
# Load optimizer and scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10)

In [43]:
# Move to GPU
model = model.to(device)
criterion = criterion.to(device)

### Train the model

In [44]:
# Main loop
best_loss = np.inf
for epoch in range(epochs):

    model.train()
    scheduler.step()
    train_loss, train_kl, train_logp = train_VAE1D(train_dl)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_kl, val_logp = train_VAE1D(val_dl)

    # Report training progress to user
    if val_loss < best_loss:
        print('Saving checkpoint..')
        best_loss = val_loss
        save_dict = {'epoch': epoch + 1,
                     'state_dict': model.state_dict(),
                     'val_loss': val_loss,
                     'optimizer': optimizer.state_dict()}
        path = save_path / f'best_model-{n_latent}-{beta}.pt'
        torch.save(save_dict, path)
        print(f'Lowest validation loss: {best_loss:.4f}')

  0%|          | 0/38 [00:00<?, ?it/s]

Saving checkpoint..
Lowest validation loss: 535.1256


  0%|          | 0/38 [00:00<?, ?it/s]

Saving checkpoint..
Lowest validation loss: 336.6423


  0%|          | 0/38 [00:00<?, ?it/s]

...

Epoch: 300 (15/38)
	Data load time: 0.148 sec
	Generation time: 0.156 sec
	Backprop time: 0.176 sec
	Log probability: -16.9168 (avg -27.6386)
	KL: 73.0461 (avg 106.0738)
	Loss: 89.9629 (avg 133.7125)


 89%|████████▉ | 34/38 [00:01<00:00, 24.04it/s]

Epoch: 300 (30/38)
	Data load time: 0.148 sec
	Generation time: 0.156 sec
	Backprop time: 0.176 sec
	Log probability: -16.6236 (avg -20.9379)
	KL: 23.3940 (avg 79.8165)
	Loss: 40.0176 (avg 100.7544)


100%|██████████| 10/10 [00:00<00:00, 28.08it/s]

VALIDATION
Epoch: 300 (10/10)
	Data load time: 0.152 sec
	Generation time: 0.160 sec
	Log probability: -14.8848 (avg -23.5989)
	KL: 21.5474 (avg 23.2306)
	Loss: 36.4322 (avg 46.8295)


# Any Fault
##### Limited data for this, only 20% of total dataset

### Build model and loss function 
Initially designed for 2D input images, modified for 1D time-series data.
Based on this paper: https://arxiv.org/abs/1807.01349

In [53]:
# The hydraulic system has 14 sensors from which to pull data
n_channels = 14
# The data has been resized to 512, although it represents 1 min for each cycle
size = 512
# Latent space is restricted to be about 1/170th of the input dims, similar to 2D case
n_latent = 50

In [54]:
model = VAE1D(size, n_channels, n_latent)

In [55]:
beta = 1  # KL term relative weight
criterion = VAE1DLoss(beta)

### Load hydraulic test data
From this dataset: https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems#

In [56]:
desc = 'any'
data_path = Path(f'data/hydraulic/{desc}/')
batch_size = 32
train_dl, val_dl, test_dl = load_datasets(data_path,
                                          batch_size=batch_size)

In [57]:
print(len(train_dl), len(val_dl), len(test_dl))

9 2 1881


### Prepare for training

In [58]:
# Display settings - TODO add Visdom logging
log_freq = 15 # batches
ep_freq = 10  # epochs

# Training parameters
epochs = 300
lr = 1e-3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Checkpoint to resume from (default None)
load_path = None

# Checkpoint save location
save_path = Path(f"models/{date.today().strftime('%y%m%d')}-{desc}/")
if save_path.is_dir():
    print(f"Folder {save_path} already exists")
else:
    os.mkdir(save_path)
save_path

PosixPath('models/190130-any')

In [59]:
# Load checkpoint if any
if load_path is not None:
    checkpoint = torch.load(load_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("Checkpoint loaded")
    print(f"Validation loss: {checkpoint['val_loss']}")
    print(f"Epoch: {checkpoint['epoch']}")

In [60]:
# Load optimizer and scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10)

In [61]:
# Move to GPU
model = model.to(device)
criterion = criterion.to(device)

### Train the model

In [62]:
# Main loop
best_loss = np.inf
for epoch in range(epochs):

    model.train()
    scheduler.step()
    train_loss, train_kl, train_logp = train_VAE1D(train_dl)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_kl, val_logp = train_VAE1D(val_dl)

    # Report training progress to user
    if val_loss < best_loss:
        print('Saving checkpoint..')
        best_loss = val_loss
        save_dict = {'epoch': epoch + 1,
                     'state_dict': model.state_dict(),
                     'val_loss': val_loss,
                     'optimizer': optimizer.state_dict()}
        path = save_path / f'best_model-{n_latent}-{beta}.pt'
        torch.save(save_dict, path)
        print(f'Lowest validation loss: {best_loss:.4f}')

  0%|          | 0/9 [00:00<?, ?it/s]

Saving checkpoint..
Lowest validation loss: 894.0638


  0%|          | 0/9 [00:00<?, ?it/s]

Saving checkpoint..
Lowest validation loss: 780.4492


  0%|          | 0/9 [00:00<?, ?it/s]

...

VALIDATION
Epoch: 290 (2/2)
	Data load time: 0.139 sec
	Generation time: 0.159 sec
	Log probability: -2.1331 (avg -2.5933)
	KL: 47.2742 (avg 1111.0459)
	Loss: 49.4072 (avg 1113.6392)


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 300 (9/9)
	Data load time: 0.149 sec
	Generation time: 0.172 sec
	Backprop time: 0.189 sec
	Log probability: -10.6175 (avg -5.9663)
	KL: 51.0887 (avg 59.1399)
	Loss: 61.7062 (avg 65.1062)


100%|██████████| 2/2 [00:00<00:00,  9.91it/s]

VALIDATION
Epoch: 300 (2/2)
	Data load time: 0.137 sec
	Generation time: 0.155 sec
	Log probability: -3.8386 (avg -3.2174)
	KL: 242.0179 (avg 148.0683)
	Loss: 245.8565 (avg 151.2857)
